Assignment 11 
Applied Machine Learning 

1. [50 pts] In this assignment, we will use a priori analysis to find phrases, or interesting word 
patterns, in a novel. 

Note that you are free to use any a priori analytics and algorithm library in this assignment. 
 
Use the nltk library corpus gutenberg API and load the novel 'carroll-alice.txt', which is Alice in Wonderland by Lewis Carroll (although his real name was Charles Dodgson). There are 1703 sentences in the novel—which can be represented as 1703 transactions. Use any means you like to parse/extract words and save in a .csv format to be read by Weka framework, similar to the a priori Analysis module. (Hint: Feel free to use mlxtend library instead of Weka.) 
 
Hint: Removing stop words and symbols using regular expressions can be helpful: 
from nltk.corpus import gutenberg, stopwords 
Stop_words = stopwords.words('english') 
Sentences = gutenberg.sents('carroll-alice.txt') 
TermsSentences = [] 
for terms in Sentences: 
  terms = [w for w in terms if w not in Stop_words] 
  terms = [w for w in terms if re.search(r'^[a-zA-Z]{2}', w) is not None] 
 
If you chose to Weka, use FPGrowth and start with default parameters. Reduce 
lowerBoundMinSupport to reach to a sweet point for the support and avoid exploding the 
number of rules generated. 
 
Report interesting patterns. 
 
(Example: Some of the frequently occurring phrases are “Mock Turtle”, “White Rabbit”, etc.) 

In [1]:
import re
import nltk
from nltk.corpus import gutenberg, stopwords
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
import pandas as pd

## DOWNLOAD NLTK DATA
nltk.download('stopwords')
nltk.download('gutenberg')
nltk.download('punkt')

## LOAD STOPWORDS + SENTENCES
stopwords = set(stopwords.words('english'))
sentences = gutenberg.sents('carroll-alice.txt')

## REMOVE STOPWORDS AND SYMBOLS VIA REGEX
processed_sentences = []
for terms in sentences:
    terms = [w.lower() for w in terms if w.lower() not in stopwords]
    terms = [re.sub(r'[^a-zA-Z]', '', w) for w in terms if re.search(r'^[a-zA-Z]{2}', w)]
    processed_sentences.append(terms)

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1091)>
[nltk_data] Error loading gutenberg: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1091)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1091)>


In [2]:
## ENCODE SENTENCES LIKE TRANSACTIONS
encoder = TransactionEncoder()
onehot = encoder.fit(processed_sentences).transform(processed_sentences)
df = pd.DataFrame(onehot, columns=encoder.columns_)

## APPLY APRIORI ALGORITHM TO GET FREQUENT ITEMS
freq_itemsets = apriori(df, min_support=0.01, use_colnames=True)
freq_itemsets.sort_values(by="support", ascending=False).head(30)

,support,itemsets
82,0.267176,(said)
1,0.228420,(alice)
128,0.095713,"(alice, said)"
51,0.068115,(little)
71,0.057546,(one)
50,0.048150,(like)
46,0.048150,(know)
110,0.048150,(went)
97,0.043453,(thought)
75,0.042866,(queen)


As shown in lecture, an itemset's support is bounded by the minimum support of any of the subsets that compose it. Thus it should be no surprise that the itemsets with the highest support are atomic, containing only one word. 

Looking at the itemsets with highest support displays the words the author uses most frequently. The most common word is "said", which is followed by "alice", the main character, and ("alice", "said") which captures all instances of alice talking. This alone can tell us that the novel is written heavily around dialogue between characters, and judging by the support values of those top 3 words, much of that dialogue is dominated by alice. 

This is interesting on its own, but doesn't tell us anything about words that are seen frequently together. There are some exceptions, like (alice, said) in the top 30, which makes sense. Phrases that combine two words



In [3]:
## LOWER THRESHOLD AND SHOW LARGER ITEMSETS
large_itemsets = apriori(df, min_support=0.001, use_colnames=True)
large_itemsets = large_itemsets[large_itemsets["itemsets"].apply(len) > 3]
large_itemsets.sort_values(by="support", ascending=False).head(30)

,support,itemsets
16945,0.002936,"(turtle, mock, said, course)"
17006,0.002936,"(little, golden, key, door)"
15851,0.002349,"(alice, said, would, know)"
17218,0.002349,"(kid, gloves, fan, white)"
17347,0.002349,"(little, glass, table, found)"
16003,0.002349,"(turtle, mock, alice, said)"
15889,0.002349,"(alice, little, said, like)"
15949,0.002349,"(tell, little, alice, said)"
16095,0.002349,"(tell, alice, would, said)"
16541,0.002349,"(ootiful, beau, oop, soo)"


Larger itemsets are much more interesting, as we can now use them to glean many more details about the characters, recurring things, and how the author's favorite words are combined. For example, (alice, would, said, know) further emphasizes that alice is speaking or otherwise narrating heavily. It is also interesting that the vast majority still contain "said", emphasizing that the novel is dialogue heavy. We also see (mock, said, gryphon, turtle), likely indicating frequent dialogue between gryphon and turtle characters. 

While relatively simple, this type of analysis already yields very interesting results on a toy problem and would be very handy when exploring new data. 

2. [50 pts] In the lecture module, the class NeuralNetMLP implements a neural network with a single hidden layer. Make the necessary modifications to upgrade it to a 2-hidden layer neural network. Run it on the MNIST dataset and report its performance. 
 
(Hint: Raschka, Chapter 12) 

In [4]:
## DATASET LOADING FUNCTION FROM THE LECTURE NOTES
def load_mnist(path, kind='train'):
    from numpy import fromfile, uint8
    import os
    import struct
    
    labels_path = os.path.join(path, '%s-labels-idx1-ubyte' % kind)
    images_path = os.path.join(path, '%s-images-idx3-ubyte' % kind)
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', lbpath.read(8))
        labels = fromfile(lbpath, dtype=uint8)
        with open(images_path, 'rb') as imgpath:
            magic, num, rows, cols = struct.unpack(">IIII",imgpath.read(16))
            images = fromfile(imgpath, dtype=uint8).reshape(len(labels), 784)
            images = ((images / 255.) - .5) * 2
    return images, labels

X_train_mnist, y_train_mnist = load_mnist('mnist', kind='train')
print(f'Rows= {X_train_mnist.shape[0]}, columns= {X_train_mnist.shape[1]}')

X_test_mnist, y_test_mnist = load_mnist('mnist', kind='t10k')
print(f'Rows= {X_test_mnist.shape[0]}, columns= {X_test_mnist.shape[1]}')

Rows= 60000, columns= 784
Rows= 10000, columns= 784


In [5]:
## MLP CLASS FROM LECTURE 
## MODIFIED WITH A SECOND HIDDEN LAYER
import numpy as np

class NeuralNetMLP(object):

    def __init__(self, n_hidden=30, epochs=100, eta=0.001, minibatch_size=1, seed=None):
        self.random = np.random.RandomState(seed)  # used to randomize weights
        self.n_hidden = n_hidden  # size of the hidden layer
        self.epochs = epochs  # number of iterations
        self.eta = eta  # learning rate
        self.minibatch_size = minibatch_size  # size of training batch - 1 would not work
        self.w_out, self.w_h1, self.w_h2 = None, None, None
    
    @staticmethod
    def onehot(_y, _n_classes):  # one hot encode the input class y
        onehot = np.zeros((_n_classes, _y.shape[0]))
        for idx, val in enumerate(_y.astype(int)):
            onehot[val, idx] = 1.0
        return onehot.T
    
    @staticmethod
    def sigmoid(_z):  # Eq 1
        return 1.0 / (1.0 + np.exp(-np.clip(_z, -250, 250)))

    def _forward(self, _X):  # Eq 2
        z_h1 = np.dot(_X, self.w_h1)
        a_h1 = self.sigmoid(z_h1)
        z_h2 = np.dot(a_h1, self.w_h2)
        a_h2 = self.sigmoid(z_h2)
        z_out = np.dot(a_h2, self.w_out)
        a_out = self.sigmoid(z_out)
        return a_h1, a_h2, a_out, z_out

    @staticmethod
    def compute_cost(y_enc, output):  # Eq 4
        term1 = -y_enc * (np.log(output))
        term2 = (1.0-y_enc) * np.log(1.0-output)
        cost = np.sum(term1 - term2)
        return cost

    def predict(self, _X):
        a_h1, a_h2, a_out, z_out = self._forward(_X)
        ypred = np.argmax(z_out, axis=1)
        return ypred

    def fit(self, _X_train, _y_train, _X_valid, _y_valid):
        import sys
        n_output = np.unique(_y_train).shape[0]  # number of class labels
        n_features = _X_train.shape[1]
        self.w_out = self.random.normal(loc=0.0, scale=0.1, size=(self.n_hidden, n_output))
        self.w_h2 = self.random.normal(loc=0.0, scale=0.1, size=(self.n_hidden, self.n_hidden))
        self.w_h1 = self.random.normal(loc=0.0, scale=0.1, size=(n_features, self.n_hidden))
        y_train_enc = self.onehot(_y_train, n_output)  # one-hot encode original y
        for ei in range(self.epochs):  # Ideally must shuffle at every epoch
            indices = np.arange(_X_train.shape[0])
            for start_idx in range(0, indices.shape[0] - self.minibatch_size + 1, self.minibatch_size):
                batch_idx = indices[start_idx:start_idx + self.minibatch_size]
                
                a_h1, a_h2, a_out, z_out = self._forward(_X_train[batch_idx])  # neural network model
                
                sigmoid_derivative_h2 = a_h2 * (1.0-a_h2)  # Eq 3
                sigmoid_derivative_h1 = a_h1 * (1.0-a_h1)  # Eq 3
                
                delta_out = a_out - y_train_enc[batch_idx]  # Eq 5
                delta_h2 = (np.dot(delta_out, self.w_out.T) * sigmoid_derivative_h2)  # Eq 6
                delta_h1 = (np.dot(delta_h2, self.w_h2.T) * sigmoid_derivative_h1)  # Eq 6
                
                grad_w_out = np.dot(a_h2.T, delta_out)  # Eq 7
                grad_w_h2 = np.dot(a_h1.T, delta_h2)  # Eq 8
                grad_w_h1 = np.dot(_X_train[batch_idx].T, delta_h1)  # Eq 8
                
                self.w_out -= self.eta*grad_w_out  # Eq 9
                self.w_h2 -= self.eta*grad_w_h2  # Eq 9
                self.w_h1 -= self.eta*grad_w_h1  # Eq 9

            # Evaluation after each epoch during training
            a_h1, a_h2, a_out, z_out = self._forward(_X_train)
            cost = self.compute_cost(y_enc=y_train_enc, output=a_out)
            y_train_pred = self.predict(_X_train)  # monitoring training progress through reclassification
            y_valid_pred = self.predict(_X_valid)  # monitoring training progress through validation
            train_acc = ((np.sum(_y_train == y_train_pred)).astype(float) / _X_train.shape[0])
            valid_acc = ((np.sum(_y_valid == y_valid_pred)).astype(float) / _X_valid.shape[0])
            sys.stderr.write('\r%d/%d | Cost: %.2f ' '| Train/Valid Acc.: %.2f%%/%.2f%% '%
                (ei+1, self.epochs, cost, train_acc*100, valid_acc*100))
            sys.stderr.flush()
        return self

In [6]:
## TRAIN THE NN
from sklearn.metrics import confusion_matrix

def get_acc(_y_test, _y_pred):
    return (np.sum(_y_test == _y_pred)).astype(float) / _y_test.shape[0]

nn = NeuralNetMLP(n_hidden=20, epochs=300, eta=0.0005, minibatch_size=100, seed=1)
nn.fit(X_train_mnist[:55000], y_train_mnist[:55000], X_train_mnist[55000:], y_train_mnist[55000:]) ;

y_pred = nn.predict(X_test_mnist)

print(f'Accuracy= {get_acc(y_test_mnist, y_pred)*100:.2f}%')
print(confusion_matrix(y_test_mnist, y_pred))

300/300 | Cost: 7190.22 | Train/Valid Acc.: 98.25%/95.96%  

Accuracy= 95.04%
[[ 950    0    0    1    4    6   14    2    3    0]
 [   0 1105    5    1    0    2    5    4   13    0]
 [   5    4  978   14    4    3    4    7   13    0]
 [   2    1   17  960    1    9    2    7    8    3]
 [   1    0    3    0  944    0    9    2    7   16]
 [  10    1    3   32    1  802   10    3   23    7]
 [   7    2    3    1    9   10  921    0    5    0]
 [   1    3   19   10    5    2    0  978    1    9]
 [   6    1    3   11    4    9    5    3  925    7]
 [   6    4    0   10   27    2    0    8   11  941]]
